# LLM Workshop
## OpenAI ChatGPT RAG

In this Notebook we are going to use ChatGPT API in combination with prompt engineering practices to implement [RAG](https://www.promptingguide.ai/techniques/rag) - Retrieval Augmented Generation 

Python code bellow makes use of the following software:
 - [OpenAI ChatGPT](https://platform.openai.com/docs/introduction) - a language model API
 - [LangChain](https://langchain.com) - a prompt engineering framework 

This notebook is also available at [Google Colab](https://colab.research.google.com/drive/11IHyhLtHb6KkXI9JjIRQA5fCjzPSE4F5?usp=sharing)

---
### Dependencies Installation

Make sure required dependencies are installed

In [ ]:
!pip install --quiet --upgrade langchain openai docarray tiktoken

---
### Import Dependencies

In [ ]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

---
### Load and inspect CSV dataset

In [ ]:
df = pd.read_csv('dataset/wine_10.csv')
df.head(10)

---
### Utility functions

The following code part contains global functions to make the RAG work.

In [ ]:
def get_api_key():
    return "?"


def get_llm(openai_api_key):
    return ChatOpenAI(
        openai_api_key=openai_api_key,
        temperature=0.0
    )


def get_csv_loader():
    return CSVLoader(file_path='dataset/wine_10.csv')


def get_embedding_model(openai_api_key):
    return OpenAIEmbeddings(
        openai_api_key=openai_api_key
    )


def get_document_retriever(embedding_model, document_loader):
    docs = document_loader.load()
    index_db = DocArrayInMemorySearch.from_documents(
        docs,
        embedding_model
    )
    return index_db.as_retriever()


def get_retrieval_qa_chain(llm_model, document_retriever):
    return RetrievalQA.from_chain_type(
        llm=llm_model,
        chain_type="stuff",
        retriever=document_retriever,
        verbose=False
    )


---
### Initialize RAG QA 

The following code block initiates a Question Answer conversation with RAG.

In [ ]:
api_key = get_api_key()
llm = get_llm(api_key)
embeddings = get_embedding_model(api_key)
csv_loader = get_csv_loader()
document_retriever = get_document_retriever(embeddings, csv_loader)
rag_qa_chain = get_retrieval_qa_chain(llm, document_retriever)

---
### Start QA session

In [ ]:
reply = rag_qa_chain.run("""
Suggest at most 3 wines that pair well with seafood, format result as a table in markdown with columns; country, title, description, variety, remark (your own remark why it goes well with seafood).
""")
display(Markdown(reply))

In [ ]:
reply = rag_qa_chain.run("""
Suggest me 3 foods that complement well Portuguese wine from the list, format result as a table in markdown with columns;
 dish_name, remark (your own remark why it goes well with suggested food), how_to_prepare (at most two sentences).
""")
display(Markdown(reply))